<a href="https://colab.research.google.com/github/shubhamksingh1/RecommendationEngine/blob/main/CollaborativeFiltering/Perspective/KNN/Collaborative_User_Based_KNN_MatrixFactorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import pandas as pd
import numpy as np
import pickle

from scipy.sparse import csr_matrix

from sklearn.neighbors import NearestNeighbors

# Reading User-rating Matrix and Asset Description Files

## Asset File

In [ ]:
ds = pd.read_csv("/content/drive/MyDrive/Collaborative_Recommendation/recommendation_v4.csv")

In [ ]:
ds.shape

(82, 8)

In [ ]:
ds.head()

,asset_id,asset_description,asset_title,custom_stop_desc,custom_stop_title,asset_description_cleaned,asset_title_cleaned,Domain
0,1,peter drucker the legendary management scholar...,the ascent of the smarter faster lighter enter...,"['3a', 'a', 'an', 'on', 'at', 'is', 's', 'do',...",['of'],peter drucker legendary management scholar con...,ascent smarter faster lighter enterprise,CX
1,2,the software development community has been us...,agile s day has truly arrived,"['a', 'u', 'we', 'k', 'i', 'on', 'at', 'is', '...",['s'],software development community using term agil...,agile day truly arrived,Agile
2,3,many exciting new digital technologies have ev...,riding the cognitive wave,"['a', 'we', 'e', 'an', 'on', 'at', 'is', 'do',...",[],many exciting new digital technology evolved r...,riding cognitive wave,AI
3,4,we have all seen some eye bulging numbers in r...,using iot data to understand how your products...,"['hp', 'a', 'we', 'an', 'on', 'at', 'is', 's',...",['to'],seen eye bulging number recent year internet t...,using iot data understand product perform,IOT
4,5,you probably need little evidence to believe t...,the view inside our take on the customer exper...,"['a', 'we', 'e', 'an', 'i', 'on', 'at', 'is', ...",['on'],probably need little evidence believe experien...,view inside take customer experience revolution,CX


In [ ]:
ds.tail()

,asset_id,asset_description,asset_title,custom_stop_desc,custom_stop_title,asset_description_cleaned,asset_title_cleaned,Domain
77,80,this report is based on tcs larger study on di...,tcs cmo study how retailers are digitally pers...,"['3a', 'a', 'we', 'on', 'at', 'is', 's', 'do',...",['to'],report based tc larger study digital marketing...,tc cmo study retailer digitally personalizing ...,Industry
78,81,this report on marketers in the travel transpo...,tcs cmo study how travel transportation and ho...,"['a', 'we', 'e', 'on', 'is', 's', 'do', 'go', ...",['to'],report marketer travel transportation hospital...,tc cmo study travel transportation hospitality...,Industry
79,83,chief marketing officers have one of the riski...,tcs cmo study stage conversion personalizing c...,"['a', 'on', 'at', 'is', 's', 'do', '■', 'vs', ...",['to'],chief marketing officer one riskiest role suit...,tc cmo study stage conversion personalizing co...,Industry
80,84,the rise of the chief medical officer typicall...,employee wellness analysis creating data drive...,"['a', 'an', 'on', 'at', 'is', 's', 'up', 'by',...",[],rise chief medical officer typically business ...,employee wellness analysis creating data drive...,HR
81,85,abstract as nations around the world deal with...,why the emotional wellbeing of your employees ...,"['a', 'we', 'an', 'on', 'is', 's', 'up', 'by',...","['a', 'of', 'be']",abstract nation around world deal managing cov...,emotional wellbeing employee top priority covid,HR


### Pickle Unpickle

In [ ]:
ds.to_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/recommendation_v4.pkl")

In [ ]:
ds = pd.read_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/recommendation_v4.pkl")

In [ ]:
ds

,asset_id,asset_description,asset_title,custom_stop_desc,custom_stop_title,asset_description_cleaned,asset_title_cleaned,Domain
0,1,peter drucker the legendary management scholar...,the ascent of the smarter faster lighter enter...,"['3a', 'a', 'an', 'on', 'at', 'is', 's', 'do',...",['of'],peter drucker legendary management scholar con...,ascent smarter faster lighter enterprise,CX
1,2,the software development community has been us...,agile s day has truly arrived,"['a', 'u', 'we', 'k', 'i', 'on', 'at', 'is', '...",['s'],software development community using term agil...,agile day truly arrived,Agile
2,3,many exciting new digital technologies have ev...,riding the cognitive wave,"['a', 'we', 'e', 'an', 'on', 'at', 'is', 'do',...",[],many exciting new digital technology evolved r...,riding cognitive wave,AI
3,4,we have all seen some eye bulging numbers in r...,using iot data to understand how your products...,"['hp', 'a', 'we', 'an', 'on', 'at', 'is', 's',...",['to'],seen eye bulging number recent year internet t...,using iot data understand product perform,IOT
4,5,you probably need little evidence to believe t...,the view inside our take on the customer exper...,"['a', 'we', 'e', 'an', 'i', 'on', 'at', 'is', ...",['on'],probably need little evidence believe experien...,view inside take customer experience revolution,CX
...,...,...,...,...,...,...,...,...
77,80,this report is based on tcs larger study on di...,tcs cmo study how retailers are digitally pers...,"['3a', 'a', 'we', 'on', 'at', 'is', 's', 'do',...",['to'],report based tc larger study digital marketing...,tc cmo study retailer digitally personalizing ...,Industry
78,81,this report on marketers in the travel transpo...,tcs cmo study how travel transportation and ho...,"['a', 'we', 'e', 'on', 'is', 's', 'do', 'go', ...",['to'],report marketer travel transportation hospital...,tc cmo study travel transportation hospitality...,Industry
79,83,chief marketing officers have one of the riski...,tcs cmo study stage conversion personalizing c...,"['a', 'on', 'at', 'is', 's', 'do', '■', 'vs', ...",['to'],chief marketing officer one riskiest role suit...,tc cmo study stage conversion personalizing co...,Industry
80,84,the rise of the chief medical officer typicall...,employee wellness analysis creating data drive...,"['a', 'an', 'on', 'at', 'is', 's', 'up', 'by',...",[],rise chief medical officer typically business ...,employee wellness analysis creating data drive...,HR


## Matrix File

Note:
We had divided users into 8 Categories as per domain derived
User         Domain


*   1 - 12 :CX
*   13 - 24 : Agile
*   25 - 36 : AI
*   37 - 48 : IOT
*   49 - 60 : BigData
*   61 - 72 : Cloud
*   73 - 84 : HR
*   85 - 100 : Industry

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Collaborative_Recommendation/rating_without_zero_v1.csv')

In [ ]:
df['Ratings'].value_counts()

3    193
4    180
5    165
2    122
1     44
Name: Ratings, dtype: int64

In [ ]:
df[df.uid==14]

,uid,asset_id,Ratings
152,14,30,1
153,14,31,1
154,14,33,5
155,14,35,1
156,14,36,3
157,14,43,4


### Pickle Unpickle

In [ ]:
df.to_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/rating_without_zero_v1.pkl")

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/rating_without_zero_v1.pkl")

In [ ]:
df[:30]

,uid,asset_id,Ratings
0,1,12,3
1,1,15,4
2,1,17,4
3,1,18,5
4,1,20,4
5,1,70,3
6,1,71,4
7,1,76,4
8,2,1,4
9,2,9,2


# Creating uid * asset_id matrix

In [ ]:
df_matrix = df.pivot(index = 'uid', columns = 'asset_id', values = 'Ratings').fillna(0)

In [ ]:
df_matrix

asset_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,59,60,61,62,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,83,84,85
uid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,4.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,2.0,4.0,2.0,4.0,0.0,0.0,3.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0,0.0,5.0,0.0,5.0,0.0,3.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,5.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,5.0,4.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,4.0,2.0,5.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,5.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,3.0,5.0,0.0,0.0


In [ ]:
type(df_matrix)

pandas.core.frame.DataFrame

## Pickle Unpickle uid *asset matrix

In [ ]:
with open('/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/uid_asset_mat.pkl', 'wb') as uam:
   pickle.dump(df_matrix, uam)

In [ ]:

df_matrix_loaded = pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/uid_asset_mat.pkl", "rb" ) )
    

In [ ]:
df_matrix_loaded

asset_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,59,60,61,62,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,83,84,85
uid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,4.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,2.0,4.0,2.0,4.0,0.0,0.0,3.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0,0.0,5.0,0.0,5.0,0.0,3.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,5.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,5.0,4.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,4.0,2.0,5.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,5.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,3.0,5.0,0.0,0.0


## Creating feature csr matrix and training KNN Model

In [ ]:
df_matrix_features = csr_matrix(df_matrix.values)

In [ ]:
type(df_matrix_features)

scipy.sparse.csr.csr_matrix

In [ ]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_jobs=-1)
model_knn.fit(df_matrix_features)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

### Pickle Unpickle KNN Model

In [ ]:
with open('/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/model_knn.pkl', 'wb') as mknn:
   pickle.dump(model_knn, mknn)

In [ ]:

model_knn_loaded = pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/model_knn.pkl", "rb" ) )
    

In [ ]:
model_knn_loaded

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

In [ ]:
model_knn

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

## Users and Asset

In [ ]:
unique_users = list(set(df['uid'].values))
unique_assets = list(set(df['asset_id'].values))

In [ ]:
len(unique_users),len(unique_assets)

(100, 82)

In [ ]:
index = unique_users.index(54)

In [ ]:
index

53

In [ ]:
distances, indices = model_knn.kneighbors(df_matrix.iloc[index, :].values.reshape(1, -1), n_neighbors = 21)

In [ ]:
distances

array([[0.        , 0.28759646, 0.3294031 , 0.34284792, 0.37293033,
        0.44714105, 0.48431795, 0.50808103, 0.55043881, 0.774087  ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        ]])

In [ ]:
indices

array([[53, 50, 55, 52, 57, 59, 49, 48, 56, 54, 70, 60, 61, 69, 65, 68,
        67, 66, 72, 64, 97]])

In [ ]:
similarities = 1-distances.flatten()

In [ ]:
similarities

array([1.        , 0.71240354, 0.6705969 , 0.65715208, 0.62706967,
       0.55285895, 0.51568205, 0.49191897, 0.44956119, 0.225913  ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [ ]:
df2 = df[['uid', 'Ratings']].groupby(['uid'], as_index = False, sort = False).mean().rename(columns = {'Ratings': 'Mean_Rating'})

In [ ]:
df2

,uid,Mean_Rating
0,1,3.875000
1,2,3.500000
2,3,3.428571
3,4,3.416667
4,5,3.636364
...,...,...
95,96,3.363636
96,97,3.642857
97,98,3.866667
98,99,3.333333


In [ ]:
df2['Mean_Rating'][97]

3.8666666666666667

### Pickle Unpickle uid Mean_Rating

In [ ]:
df2.to_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/user_mean_rating.pkl")

In [ ]:
df2 = pd.read_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/user_mean_rating.pkl")

In [ ]:
df2

,uid,Mean_Rating
0,1,3.875000
1,2,3.500000
2,3,3.428571
3,4,3.416667
4,5,3.636364
...,...,...
95,96,3.363636
96,97,3.642857
97,98,3.866667
98,99,3.333333


# Recommendation

In [ ]:
def find_similar_users(index):
    distances, indices = model_knn.kneighbors(df_matrix.iloc[index, :].values.reshape(1, -1), n_neighbors = 21)
    similarities = 1 - distances.flatten() #Converting to 1-Dimention and increasing score for nearer users
    
    return similarities, indices

In [ ]:
def predict_rating(user_id, asset_id):
    unique_users = list(set(df['uid'].values))
    user_index = unique_users.index(user_id)
    unique_assets = list(set(df['asset_id'].values))
    asset_index = unique_assets.index(asset_id)
    similarities, indices = find_similar_users(user_index)
    mean_rating = df2['Mean_Rating'][user_index]
    #mean_rating = df_matrix.iloc[user_index,:].mean()
    sum_wt = np.sum(similarities) - 1
    weighted_sum = 0
    prediction = 0
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == user_index:
            continue;
        elif (df_matrix.iloc[indices.flatten()[i],asset_index]) != 0:
            j = indices.flatten()[i] 
            ratings_diff = df_matrix.iloc[j, asset_index] - df2['Mean_Rating'][j]#np.mean(df_matrix.iloc[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            weighted_sum = weighted_sum + product
    prediction = mean_rating + (weighted_sum/sum_wt)
    
    return prediction

In [ ]:
predict_rating(1, 2)

3.875

In [ ]:
indices_n = pd.Series(ds['asset_id'])

In [ ]:
print(indices_n)

0      1
1      2
2      3
3      4
4      5
      ..
77    80
78    81
79    83
80    84
81    85
Name: asset_id, Length: 82, dtype: int64


In [ ]:

inddict = indices_n.to_dict()

In [ ]:
print(inddict)

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 55: 56, 56: 57, 57: 59, 58: 60, 59: 61, 60: 62, 61: 63, 62: 65, 63: 66, 64: 67, 65: 68, 66: 69, 67: 70, 68: 71, 69: 72, 70: 73, 71: 74, 72: 75, 73: 76, 74: 77, 75: 78, 76: 79, 77: 80, 78: 81, 79: 83, 80: 84, 81: 85}


In [ ]:

#changing the selection of assets from index to asset_id
inddict = dict((v,k) for k,v in inddict.items())

In [ ]:
print(inddict)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 59: 57, 60: 58, 61: 59, 62: 60, 63: 61, 65: 62, 66: 63, 67: 64, 68: 65, 69: 66, 70: 67, 71: 68, 72: 69, 73: 70, 74: 71, 75: 72, 76: 73, 77: 74, 78: 75, 79: 76, 80: 77, 81: 78, 83: 79, 84: 80, 85: 81}


## Pickle Unpickle Inddict

In [ ]:
with open('/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/inddict.pkl', 'wb') as ind:
   pickle.dump(inddict, ind)

In [ ]:

inddict_loaded = pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/inddict.pkl", "rb" ) )
    

In [ ]:
print(inddict_loaded)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 59: 57, 60: 58, 61: 59, 62: 60, 63: 61, 65: 62, 66: 63, 67: 64, 68: 65, 69: 66, 70: 67, 71: 68, 72: 69, 73: 70, 74: 71, 75: 72, 76: 73, 77: 74, 78: 75, 79: 76, 80: 77, 81: 78, 83: 79, 84: 80, 85: 81}


In [ ]:
def get_recommendations(user_id, n):
    n = n + 1
    assets = []
    ratings = []
    unique_assets = list(set(df['asset_id'].values))
    asset_rated_by_user = df.loc[df['uid'] == user_id, 'asset_id'] # Assets of which user 'user_id' already rated
    if len(asset_rated_by_user) >= 3:
        assets_to_pred = np.setdiff1d(unique_assets, asset_rated_by_user) # Remove assets that 'user_id' has rated from list of all assets
        for i in assets_to_pred:
            asset_rating = predict_rating(user_id, i) 
            ratings.append(asset_rating)
        pred_ratings = np.array(ratings)
        indices = pred_ratings.argsort()[:-n:-1]
        for i in indices:
            assets.append(assets_to_pred[i])
        iids = [inddict[asset_id] for asset_id in assets]
        ds2 = ds.loc[:, ['asset_id', 'asset_title', 'asset_description', 'Domain']]
        return ds2.iloc[iids]
    else:
        return 'User has rated less than 3 articles'
    

In [ ]:
ds.columns

Index(['asset_id', 'asset_description', 'asset_title', 'custom_stop_desc',
       'custom_stop_title', 'asset_description_cleaned', 'asset_title_cleaned',
       'Domain'],
      dtype='object')

Users      Domain
*   1 - 12 :CX
*   13 - 24 : Agile
*   25 - 36 : AI
*   37 - 48 : IOT
*   49 - 60 : BigData
*   61 - 72 : Cloud
*   73 - 84 : HR
*   85 - 100 : Industry

In [ ]:
ds.Domain.value_counts()

Industry    21
CX          19
Agile       10
AI           8
Cloud        7
IOT          6
BigData      6
HR           5
Name: Domain, dtype: int64

In [ ]:
get_recommendations(1, 5)

,asset_id,asset_title,asset_description,Domain
0,1,the ascent of the smarter faster lighter enter...,peter drucker the legendary management scholar...,CX
13,14,when cx should begin with tweaks not disruption,today s most over used corporate cliché is dis...,CX
71,74,proactive protection how companies can secure ...,today digital customer experience transformati...,CX
40,41,the magic that happens when manufacturers digi...,a growing number of manufacturers are digitall...,IOT
39,40,now you can simulate nearly anything,organizations have long performed what if simu...,Cloud


In [ ]:
get_recommendations(14, 5)

,asset_id,asset_title,asset_description,Domain
62,65,an agile manifesto for the board,it s hard to avoid the term agile these days t...,Agile
1,2,agile s day has truly arrived,the software development community has been us...,Agile
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
25,26,training for the m a marathon,a few years ago two airlines announced a merge...,Industry
19,20,reimagining the contact center with digital te...,it costs billion per year to field the billion...,CX


In [ ]:
get_recommendations(28, 5)

,asset_id,asset_title,asset_description,Domain
57,59,taking a machine first approach to digital tra...,many companies believe they can compete in the...,AI
66,69,how intelligent automation is rewriting the ru...,customer support has become a key differentiat...,AI
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
21,22,why your cx must soon plug and play into a dig...,digital ecosystems increasingly are coalescing...,CX
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud


In [ ]:
get_recommendations(47,5)

,asset_id,asset_title,asset_description,Domain
3,4,using iot data to understand how your products...,we have all seen some eye bulging numbers in r...,IOT
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
29,30,embracing agility means agility by the busines...,the push for agility in software development s...,Agile
22,23,what comes after cxm helping customers help ot...,thirty years ago large global companies began ...,CX
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud


In [ ]:
get_recommendations(55,5)

,asset_id,asset_title,asset_description,Domain
54,55,digital data s greatest opportunities,every company today must digitally transform i...,BigData
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
29,30,embracing agility means agility by the busines...,the push for agility in software development s...,Agile
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud
24,25,how the ceo can propel a cx initiative,whether it s a series of tweaks or a wholesale...,CX


In [ ]:
get_recommendations(63,5)

,asset_id,asset_title,asset_description,Domain
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
29,30,embracing agility means agility by the busines...,the push for agility in software development s...,Agile
21,22,why your cx must soon plug and play into a dig...,digital ecosystems increasingly are coalescing...,CX
22,23,what comes after cxm helping customers help ot...,thirty years ago large global companies began ...,CX


In [ ]:
get_recommendations(74,5)

,asset_id,asset_title,asset_description,Domain
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
30,31,how to make location independent agile work,while it is very beneficial to have all the te...,Agile
22,23,what comes after cxm helping customers help ot...,thirty years ago large global companies began ...,CX
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud
24,25,how the ceo can propel a cx initiative,whether it s a series of tweaks or a wholesale...,CX


In [ ]:
get_recommendations(80,5)

,asset_id,asset_title,asset_description,Domain
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
28,29,students for life why executives must continua...,companies today must get smarter faster and li...,HR
29,30,embracing agility means agility by the busines...,the push for agility in software development s...,Agile
22,23,what comes after cxm helping customers help ot...,thirty years ago large global companies began ...,CX
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud


In [ ]:
get_recommendations(83,5)

'User has rated less than 3 articles'

In [ ]:
get_recommendations(95,5)

,asset_id,asset_title,asset_description,Domain
64,67,unleashing business value in a business world,in an era we call business digital technologie...,Industry
55,56,how the digital renovators are racing ahead,numerous studies show that the majority of lar...,Industry
79,83,tcs cmo study stage conversion personalizing c...,chief marketing officers have one of the riski...,Industry
38,39,why your products must be smart and connected,it s become increasingly conceivable and cost ...,Industry
51,52,from making autos to making software the evolu...,michael a cusumano1 is a professor at the mass...,Industry
